<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/Onnx_and_fine_tuned__predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install transformers transformers[onnx]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import libraries

In [ ]:
import torch
import onnxruntime as ort
import os
from transformers import AlbertForMaskedLM, RobertaTokenizerFast

## Predictions using ONNX model

In [ ]:
# Load the fine-tuned model and tokenizer
output_dir = '/content/drive/MyDrive/fine-tuned-albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = RobertaTokenizerFast.from_pretrained(output_dir)

# Export the model to ONNX format
onnx_path = "/content/drive/MyDrive/fine-tuned-albert/fine_tuned_model.onnx"
input_names = ["input_ids", "attention_mask"]
output_names = ["output"]
dynamic_axes = {
    "input_ids": {0: "batch_size", 1: "sequence_length"},
    "attention_mask": {0: "batch_size", 1: "sequence_length"},
    "output": {0: "batch_size", 1: "sequence_length"}
}
torch.onnx.export(
    model,
    (
        torch.zeros([1, 1], dtype=torch.long),  # Placeholder input tensor for input_ids
        torch.zeros([1, 1], dtype=torch.long),  # Placeholder input tensor for attention_mask
    ),
    onnx_path,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes=dynamic_axes,
    opset_version=17
)

# Print the ONNX model path
print(f"ONNX model saved at: {onnx_path}")


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX model saved at: /content/drive/MyDrive/fine-tuned-albert/fine_tuned_model.onnx


In [ ]:
import onnxruntime as ort
import torch

# Load the tokenizer
output_dir = '/content/drive/MyDrive/fine-tuned-albert'
tokenizer = RobertaTokenizerFast.from_pretrained(output_dir)

# Load the ONNX model
onnx_path = "/content/drive/MyDrive/fine-tuned-albert/fine_tuned_model.onnx"
ort_session = ort.InferenceSession(onnx_path)

# Example masked sentence
masked_sentence = "int <mask> = getCount();"

# Tokenize the masked sentence
inputs = tokenizer.encode_plus(masked_sentence, add_special_tokens=False, return_tensors='pt')

# Extract input tensors
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Run the ONNX model
outputs = ort_session.run(None, {
    "input_ids": input_ids.numpy(),
    "attention_mask": attention_mask.numpy(),
})

# Get the predictions
predictions = outputs[0]

# Find the position of the masked token
masked_token_index = input_ids[0].tolist().index(tokenizer.mask_token_id)

# Get the top predicted tokens
top_k = torch.topk(torch.softmax(torch.tensor(predictions[0, masked_token_index]), dim=-1), k=5)

# Decode and print the top predicted tokens
print("ONNX Model Predictions:")
for i, idx in enumerate(top_k.indices):
    token = tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '')
    prob = top_k.values[i].item()
    print(f"{token}")


ONNX Model Predictions:
check
m
result
res
binding


In [ ]:
import torch
from transformers import AlbertForMaskedLM, RobertaTokenizerFast

def get_predictions(input_statement):
    # Load the trained model and tokenizer
    output_dir = '/content/drive/MyDrive/fine-tuned-albert'
    model = AlbertForMaskedLM.from_pretrained(output_dir)
    tokenizer = RobertaTokenizerFast.from_pretrained(output_dir)

    # Tokenize the masked statement
    input_ids = tokenizer.encode(input_statement, add_special_tokens=False, return_tensors='pt')

    # Find the position of the masked token
    masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

    # Generate predictions for the masked token using the fine-tuned model
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs[0]

    # Get the top 5 predictions and their probability scores from the fine-tuned model
    probs_ft = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k_ft = torch.topk(probs_ft, k=5)

    # Print the top predictions and their probabilities
    print("Fine-Tuned Model Predictions:")
    for idx in top_k_ft.indices:
        token = tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower()
        prob = probs_ft[idx].item()
        print(f"{token}")

# Example usage
input_statement = "int <mask> = getCount();"
get_predictions(input_statement)


Fine-Tuned Model Predictions:
check
m
result
res
binding
